In [0]:
dbutils.widgets.text("p_data_source", "")
v_data_source = dbutils.widgets.get("p_data_source")

In [0]:
dbutils.widgets.text("p_file_date", "2021-03-21")
v_file_date = dbutils.widgets.get("p_file_date")

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/common_functions"

In [0]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DateType

In [0]:
pit_stops_schema = StructType(fields=[
    StructField("raceId", IntegerType(), True),
    StructField("driverId", IntegerType(), True),
    StructField("stop", IntegerType(), True),
    StructField("lap", IntegerType(), True),
    StructField("time", StringType(), True),
    StructField("duration", StringType(), True),
    StructField("milliseconds", IntegerType(), True)
])

In [0]:
pit_stops_df = spark.read \
.schema(pit_stops_schema) \
.csv(f"{raw_folder_path}/{v_file_date}/pit_stops.csv") 



In [0]:
from pyspark.sql.functions import lit

In [0]:
pit_stops_final_df = add_ingestion_date(pit_stops_df) \
    .withColumnRenamed("raceId", "race_id") \
.withColumnRenamed("driverId", "driver_id") \
.withColumn("data_source", lit(v_data_source)) \
    .withColumn("file_date", lit(v_file_date))

In [0]:
merge_condition = "tgt.driver_id = src.driver_id AND tgt.race_id = src.race_id AND tgt.stop = src.stop"
merge_delta_data(pit_stops_final_df,'f1_processed', 'pit_stops', merge_condition,'race_id')

In [0]:
dbutils.notebook.exit("success")